In [1]:
import numpy as np
import pandas as pd
import random
import copy
import collections

In [2]:
class My_louvain:
    def __init__(self, k):  # k:节点个数
        self.k = k
        
    def get_data(self):
        # np.random.seed(10)
        data = np.zeros((self.k, self.k))
        for i in range(len(data)):
            for j in range(len(data)):
                data[i,j] = np.random.choice([0,1], p=[0.9, 0.1])
        data = data + np.multiply(data.T, data.T > data) - np.multiply(data, data.T > data)
        for i in range(len(data)):
            data[i, i] = 0
        data = pd.DataFrame(data)
        return data

    # def get_data(self):
    #     data = np.array([[0,1,1,1,0,0,0,0,0,0],
    #                     [1,0,1,0,0,0,0,0,0,0],
    #                     [1,1,0,1,0,0,0,0,0,0],
    #                     [1,0,1,0,1,1,1,0,0,0],
    #                     [0,0,0,1,0,1,0,0,0,0],
    #                     [0,0,0,1,1,0,1,0,0,0],
    #                     [0,0,0,1,0,1,0,0,0,0],
    #                     [0,0,0,0,0,0,0,0,1,1],
    #                     [0,0,0,0,0,0,0,1,0,1],
    #                     [0,0,0,0,0,0,0,1,1,0]])
    #     data = pd.DataFrame(data)
    #     return data
    
    def Q_delta_i(self, sum_in, sum_tot, ki, ki_in, m, direction):
        Q_before_i = (sum_in / 2 / m - (sum_tot / 2 / m) ** 2) + (0 - (ki / 2 / m) ** 2)
        Q_after_i = (sum_in + ki_in) / 2 / m - ((sum_tot + ki) / 2 / m) ** 2

        if direction == "di":
            Q_delta_i = Q_before_i - Q_after_i
        if direction == "ic":
            Q_delta_i = Q_after_i - Q_before_i
        return Q_delta_i

    def Q_delta_dic(self, Q_delta_ic, Q_delta_di):
        Q_delta = Q_delta_ic + Q_delta_di
        return Q_delta


    def cal_Q_delta(self, data, community_D, community_C): # 从D社区中拿出节点，放入C社区中。community_D:list community_C:list
        """ 
        将 D 社区中的节点提取出来 放入 C 社区中, 需要分别计算将 node 从 D 社区 提取出来的Q_delta_di 和 将 node 放入 C社区的 Q_delta_ic
        并且计算两个Q_delta 所用的 sum_in, sum_tot, ki, ki_in 是根据 node 的不同而改变的。具体而言: 把 node 从 D 社区提取出来时构造一个剔除 node 的 temp_list,
        计算 node 对 temp_list 的四个统计数据, 把 node 放入 C 社区时计算 node 对 C 社区内所有节点的四个统计数据. 最终构造的字典 node 为 key Q_delta 为 value

        注: 这里 community_D 和 community_C 是单个社区不同于下面的社区群组 community_Ds community_Cs
        """
        m = data.sum().sum() / 2
        Q_delta_dict = {}
        for i in range(len(community_D)):
            # 这里计算 将 i 节点 从 D 社区中拿出来的 一系列描述性统计数据以及 Q_delata_di
            node_i = community_D[i]
            temp_list = community_D[:i] + community_D[i+1:]    # 这个temp存储的是除了 i 节点外剩下的社区
            sum_in_di = data.loc[temp_list, temp_list].sum().sum()
            sum_tot_di = data.loc[temp_list,:].sum().sum()
            ki = data.loc[node_i,:].sum()
            ki_in_di = np.sum(np.sum(data.loc[node_i, temp_list])) * 2
            Q_delta_di = self.Q_delta_i(sum_in_di, sum_tot_di, ki, ki_in_di, m, "di")

            # 这里开始计算将 i 节点 放入 C 社区中的 一系列描述性统计数据以及Q_delta_ic
            sum_in_ic = data.loc[community_C, community_C].sum().sum()
            sum_tot_ic = data.loc[community_C, community_C + [node_i]].sum().sum()
            ki_in_ic = data.loc[community_C, node_i].sum()
            Q_delta_ic = self.Q_delta_i(sum_in_ic, sum_tot_ic, ki, ki_in_ic, m, "ic")
            
            # Q_delta_dic
            Q_delta = self.Q_delta_dic(Q_delta_ic, Q_delta_di)
            Q_delta_dict[node_i] = Q_delta
        # print(Q_delta_dict)
        return Q_delta_dict

  

    def phase_one(self,data, community_Ds, community_Cs):  
        """
        community_Ds存放所有D社区, community_Cs存放所有C社区, 对于一个 commmunity_D 都计算将 community_D 中的每一个节点加入一个community_C中的Q_delta_dict
        并且以该 community_C的索引为 key Q_delta_dict 为 value构造一个嵌套字典,转化成 dataframe index 为一个 community_D中的每一个节点, collumns 为community_Cs
        中的每一个 community_C 根据 dataframe 方便的找出具体是 将 node 加入哪一个 community_C 中获得的增益 Q_delta 最大
        对于该增益最大的 community_C 将 node 加入其中 
        """
        mod_inc = False
        while True:
            can_stop = True
            for community_D in community_Ds:
                # 根据计算的 Q_delta 构造一个 dataframe
                Q_delta_dicts = {}
                for i in range(len(community_Cs)):
                    if community_D == community_Cs[i]:
                        pass
                    else:
                        Q_delta_dict = self.cal_Q_delta(data, community_D, community_Cs[i])
                        Q_delta_dicts[i] = Q_delta_dict
                    # print(Q_delta_dict)
                df = pd.DataFrame(Q_delta_dicts, columns=Q_delta_dicts.keys())    # index 为一个 community_D中的每一个节点, collumns 为community_Cs中的每一个 community_C
                # print(df)
                # 根据构建的 dataframe 确定将 D 社区中的节点加入 具体某一个C中,即放入增益最大的社区C中
                for i in reversed(range(len(df.index))):
                    node = df.index[i]
                    row = df.loc[node,:]
                    Q = row.max()
                    # print("Q",Q)
                    community = row[row==row.max()].index
                    # community = sorted(row[row == row.max()].index)[0]
                    community = np.random.choice(community)  # 当有多个社区的增益相同时, 随机挑选一个
                    
                    if Q > 0:
                        for community_C in community_Cs:
                            if node in community_C:
                                community_C.remove(node)            
                        community_Cs[community].append(node)
                        community_Ds = community_Cs
                        can_stop = False
                        mod_inc = True
                    # print(mod_inc)

            print(community_Cs)
            if can_stop:
                break
        # print(mod_inc)               
            # 从 community_Cs 中删除节点时有可能会产生"空社区"(即该社区中的所有节点都转移到了其他社区,变成了一个空社区), 删除"空社区",得到phase_one的社区
        while [] in community_Cs:
            community_Cs.remove([])
        return community_Cs, mod_inc
    
    def execute_phase_one(self, data, community_Cs):
        community_Ds = community_Cs
        community_Cs, mod_inc = self.phase_one(data, community_Ds, community_Cs)
        # print(community_Cs)
        # print(mod_inc)
        return community_Cs, mod_inc


    def phase_two(self, data, community_Cs):
        large_node = {}
        for i in range(len(community_Cs)):
            # community_name = np.sum(community_Cs[i])
            community_name = i
            # print(community_name)
            large_node[community_name] = community_Cs[i]
        new_data = pd.DataFrame(index = large_node.keys(), columns = large_node.keys())
        for i in new_data.index:
            for j in new_data.columns:
                new_data.loc[i,j] = np.sum(np.sum(data.loc[large_node[i], large_node[j]]))
        return new_data, large_node

    def execute(self):
        iter_times = 0
        data = self.get_data()
        community_Cs = [[i] for i in range(len(data.columns))]
        community_Ds = copy.deepcopy(community_Cs)
        while True:
            iter_times += 1
            community_Cs, mod_inc = self.phase_one(data, community_Ds, community_Cs)
            # print(community_Cs)
            if mod_inc:
                data, large_node = self.phase_two(data, community_Cs)
                print("iter_times:", iter_times, large_node)
                community_Cs = [[i] for i in large_node.keys()]
                community_Ds = copy.deepcopy(community_Cs)
                # last_community = 
            else:
                break

            
    # def execute_phase(self):
    #     data = self.get_data()
    #     community_Cs = [[i] for i in data.columns]
    #     random.shuffle(community_Cs)


        # community_Ds = copy.deepcopy(community_Cs)
        # community = self.phase_one(data, community_Ds, community_Cs)
        # # print(community)
        # return community_Cs

    # def execute_all_phase(self):
    #     # np.random.seed(10)
    #     data = ml.get_data()
    #     community_Cs = [[i] for i in data.columns]
    #     random.shuffle(community_Cs)
    #     for i in range(self.iterations):
    #         community_Cs = self.execute_phase_one(data, community_Cs)
    #         print("第{}次,phase_one".format(i), community_Cs)
    #         community_Cs = self.phase_two(data, community_Cs)
    #         print("第{}次,phase_two".format(i), community_Cs)
    #     return data

In [3]:
ml = My_louvain(k = 50)
ml.execute()

[[], [], [], [], [], [], [], [], [], [], [], [5, 47], [7, 14], [2, 21], [9, 36], [], [4, 32, 42], [], [12, 38, 40], [0, 11, 13, 19, 22, 27, 30, 37, 41, 43], [], [], [], [], [], [16, 29], [3, 24], [], [], [], [], [18], [], [15, 23, 33], [], [], [], [20], [1], [6, 8, 26, 28, 31, 39, 44], [25], [], [], [35], [], [], [], [], [17], [10, 34, 45, 46, 48, 49]]
[[], [], [], [], [], [], [], [], [], [], [], [5, 47], [], [2, 21], [36], [], [32, 42], [], [12, 38, 40, 7], [0, 11, 13, 19, 22, 27, 30, 41, 43], [], [], [], [], [], [16, 29], [3, 24, 20, 35], [], [], [], [], [], [], [15, 23, 33, 4, 1], [], [], [], [], [], [6, 8, 26, 28, 31, 39, 44, 9, 18, 25], [], [], [], [], [], [], [], [], [], [10, 34, 45, 46, 48, 49, 14, 37, 17]]
[[], [], [], [], [], [], [], [], [], [], [], [5, 47], [], [2, 21, 36], [], [], [32, 42], [], [12, 38, 40, 7], [0, 11, 13, 19, 22, 27, 30, 41, 43], [], [], [], [], [], [16, 29], [3, 24, 20, 35], [], [], [], [], [], [], [15, 23, 33, 4, 1], [], [], [], [], [], [6, 8, 26, 28, 31,